In [1]:
library(verification)
library(kableExtra)

setwd('../../../')
getwd()

Loading required package: fields

Loading required package: spam

Spam version 2.11-1 (2025-01-20) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.


Attaching package: ‘spam’


The following objects are masked from ‘package:base’:

    backsolve, forwardsolve


Loading required package: viridisLite


Try help(fields) to get started.

Loading required package: boot

Loading required package: CircStats

Loading required package: MASS

Loading required package: dtw

Loading required package: proxy


Attaching package: ‘proxy’


The following object is masked from ‘package:spam’:

    as.matrix


The following objects are masked from ‘package:stats’:

    as.dist, dist


The following object is masked from ‘package:base’:

    as.matrix


Loaded dtw v1.23-1. See ?dtw for help, citation("dtw") for use in publication

[1] "/home/carla/Documents/keep_trying/WindForecastEval"

In [2]:
path_in = normalizePath(paste0(getwd(), '/ipynb/probabilistic_forecasts/bs/rds'))

names_data <- list.files(path = path_in,
                         pattern = '^terciles.*\\.rds$',
                         full.names = T)

list_data <- lapply(names_data, readRDS)

names(list_data) <- list.files(path = path_in,
                               pattern = '^terciles.*\\.rds$',
                               full.names = F)

 [1] "median_prob_and_reference_cfsv2_djf_r1.rds"  
 [2] "median_prob_and_reference_cfsv2_djf_r2.rds"  
 [3] "median_prob_and_reference_cfsv2_djf_r3.rds"  
 [4] "median_prob_and_reference_seas5_djf_r1.rds"  
 [5] "median_prob_and_reference_seas5_djf_r2.rds"  
 [6] "median_prob_and_reference_seas5_djf_r3.rds"  
 [7] "terciles_prob_and_reference_cfsv2_djf_r1.rds"
 [8] "terciles_prob_and_reference_cfsv2_djf_r2.rds"
 [9] "terciles_prob_and_reference_cfsv2_djf_r3.rds"
[10] "terciles_prob_and_reference_seas5_djf_r1.rds"
[11] "terciles_prob_and_reference_seas5_djf_r2.rds"
[12] "terciles_prob_and_reference_seas5_djf_r3.rds"


In [12]:
data_process = function(prob, categoria, tercile){
  probabilidad = as.vector(prob[tercile,,])
  categoria = as.vector(categoria)
  cat_total = numeric(length(categoria))
  cat_total[categoria==tercile] = 1
  result = data.frame(pred = probabilidad, obs =  cat_total)
  colnames(result) = c('pred', 'obs')
  
  
  return(result)
}


bs.result <- function(list_data, tercile, thresh = seq(0,1,0.25)){
  data_tercile1 = list()
  j <- 0
  names <- names(list_data)
  
  # Roc.area result
  bs_tercile1 <- list()
  
  for (i in list_data){
    j <- j + 1
    
    partial <- data_process(prob = i$prob, categoria = i$obs, tercile = tercile)
    
    data_tercile1[[names[j]]] <- partial
    
    bs_tercile1[[names[j]]] <- brier(obs = partial$obs, 
                                     pred = partial$pred,
                                     bins = T,
                                     thresholds = thresh)
  }
  
  result_bs_tercile1 <- data.frame(bs =  sapply(bs_tercile1, `[[`, 'bs'),
                                   ss =  sapply(bs_tercile1, `[[`, 'ss'),
                                   bs.reliability	= sapply(bs_tercile1, `[[`, 'bs.reliability'),
                                   bs.resolution = sapply(bs_tercile1, `[[`, 'bs.resol'),
                                   bs.uncert = sapply(bs_tercile1, `[[`, 'bs.uncert'),
                                   check = sapply(bs_tercile1, `[[`, 'check'),
                                   bins = sapply(bs_tercile1, `[[`, 'bins'))
  
  return(result_bs_tercile1)
}

rsult_t1_bs <- bs.result(list_data = list_data, tercile = 1)
rsult_t2_bs <- bs.result(list_data = list_data, tercile = 2)
rsult_t3_bs <- bs.result(list_data = list_data, tercile = 3)


In [13]:
rsult_t1_bs %>%
  kable("html", caption =  "Table of Brier Score Results (Below Normal/Lower Tercile)") %>%
  kable_styling(bootstrap_options = c("striped", "hover"), full_width = FALSE) 

<table class="table table-striped table-hover" style="width: auto !important; margin-left: auto; margin-right: auto;">
<caption>Tabla de Resultados Brier Score (Tercil Inferior)</caption>
 <thead>
  <tr>
   <th style="text-align:left;">   </th>
   <th style="text-align:right;"> bs </th>
   <th style="text-align:right;"> ss </th>
   <th style="text-align:right;"> bs.reliability </th>
   <th style="text-align:right;"> bs.resolution </th>
   <th style="text-align:right;"> bs.uncert </th>
   <th style="text-align:right;"> check </th>
   <th style="text-align:left;"> bins </th>
  </tr>
 </thead>
<tbody>
  <tr>
   <td style="text-align:left;"> terciles_prob_and_reference_cfsv2_djf_r1.rds </td>
   <td style="text-align:right;"> 0.1944444 </td>
   <td style="text-align:right;"> 0.1421277 </td>
   <td style="text-align:right;"> 0.0032768 </td>
   <td style="text-align:right;"> 0.0354913 </td>
   <td style="text-align:right;"> 0.2266590 </td>
   <td style="text-align:right;"> 0.1944444 </td>
   

In [ ]:
rsult_t2_bs %>%
  kable("html", caption =  "Table of Brier Score Results (Normal)") %>%
  kable_styling(bootstrap_options = c("striped", "hover"), full_width = FALSE) 

In [ ]:
rsult_t3_bs %>%
  kable("html", caption =  "Table of Brier Score Results (Above Normal/Upper Tercile)") %>%
  kable_styling(bootstrap_options = c("striped", "hover"), full_width = FALSE) 